# MLGeometry guide

This introduction demonstrates how to use MLGeometry to:
1. Generate a hypersurface.
2. Build a bihomogeneous neural network.
3. Use the model to compute numerical Calabi-Yau metrics with the embedding method.
4. Plot $\eta$ on a rational curve.

## Install the package (on Colab)

In [ ]:
!pip install MLGeometry-tf

## Configure imports

Import tensorflow_probability to use the L-BFGS optimizer:

In [ ]:
import sympy as sp
import tensorflow as tf
import tensorflow_probability as tfp
import keras
import csv
from google.colab import files

In [ ]:
import MLGeometry as mlg
from MLGeometry import bihomoNN as bnn

Import the libraries to plot the $\eta$ on the rational curve (see the last section):

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Set a random seed (optional)
Some random seed might be bad for numerical calulations. If there are any errors during the training, you may want to try a different seed.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

## Define a hypersurface
First define a set of coordinates and a function as sympy symbols:

In [ ]:
# Elliptic curves as CY 1-fold in CP^2
z0, z1, z2 = sp.symbols('z0, z1, z2')

Z = [z0, z1, z2]

fermat = z0**3 + z1**3 + z2**3

zeta5 = np.exp(2*np.pi*1j/5)

t = 0.1

# Dwork family (indexed by t) in CP^3
f_x = fermat + t*z0*z1*z2  # X_t
"""
#f_z = t*fermat + (z1**4 + z2**4 + z3**4) + z0*(z1**3 + z2**3 + z3**3) + z0**3 * (z1 + z2 + z3)  # Z_t (wrong equation)
f_z = t*fermat + (z1**4 + z2**4 + z3**4) + z0*(z1**3 + z2**3 + z3**3) + z0**2 * (z1**2 + z2**2 + z3**2)  # Z_t
#f_w = t*fermat + (z0**2 + z1*z2 + z3**2)*(z1**2 + z2*z3 + z0**2)  # W_t
f_w = t*fermat + z0*(z0**3 + z1**3 + z2**3 + z3**3) # W_t
f_w2 = t*fermat + z0*(z1**3 + z2**3 + z3**3)
"""
f_v = t*fermat + z0*z1*z2 # V_t

f = f_v

Then define a hypersurface as a collection of points which solve the equation f = 0, using the `Hypersurface` class in the `mlg.hypersurface` module. The parameter n_pairs is the number of random pairs of points used to form the random lines in $\mathbf{CP}^{N+1}$. Then we take the intersections of those random lines and the hypersurface. By Bezout's theorem, each line intersects the hypersurface in precisely d points where d is the number of homogeneous coordinates. So the total number of points is d * n_pairs.

In [ ]:
n_pairs = 1280
HS_train = mlg.hypersurface.Hypersurface(Z, f, n_pairs)
HS_test = mlg.hypersurface.Hypersurface(Z, f, n_pairs)

The Hypersurface class will take care of the patchwork automatically. Let's use the `list_patches` function to check the number of points on each patch:

In [ ]:
HS_train.list_patches()

Number of Patches: 3
Points on patch 1 : 1270
Points on patch 2 : 1286
Points on patch 3 : 1284


You can also invoke this method on one of the patches to check the distribution on the subpatches:

In [ ]:
HS_train.patches[0].list_patches()

Number of Patches: 2
Points on patch 1 : 644
Points on patch 2 : 626


The Hypersurface class contains some symbolic and numerical methods as well, which will be introduced elsewhere.

## Training with Tensorflow
The following steps are similar to a regular Tensorflow training process.
### Generate datasets
The `mlg.tf_dataset.generate_dataset` function converts a hypersurface to a Tensorflow Dataset, which has four componets: the points on the hypersurface, the volume form $\small \Omega \wedge \bar\Omega$, the mass reweighting the points distribution and the restriction which restricts the Kähler metric to a subpatch. The restriction contains an extra linear transformation so that points on different affine patches can all be processed in one call. It is also possible to generate a dataset only on one affine patch.

In [ ]:
train_set = mlg.tf_dataset.generate_dataset(HS_train)
test_set = mlg.tf_dataset.generate_dataset(HS_test)

Shuffle and batch the datasets:

In [ ]:
train_set = train_set.shuffle(HS_train.n_points).batch(128)
test_set = test_set.shuffle(HS_test.n_points).batch(128)

Let's look at what is inside a dataset:

In [ ]:
points, Omega_Omegabar, mass, restriction = next(iter(train_set))
print(points[1])
pts = points.numpy()

tf.Tensor([-0.06983887-0.06357283j  1.        +0.j          0.42707974-0.5707137j ], shape=(3,), dtype=complex64)


### Build a bihomogeneous neural network

The `mlg.bihomoNN` module provides the necessary layers (e.g. `Bihomogeneous` and `Dense` ) to construct the Kähler potential with a bihomogeneous neural network. Here is an example of a two-hidden-layer network (k = 4) with 70 and 100 hidden units:

In [ ]:
@keras.saving.register_keras_serializable(package="MLGeometry")
class Kahler_potential(tf.keras.Model):
    def __init__(self, trainable=True, dtype='float32', **kwargs):
        super(Kahler_potential, self).__init__(trainable=trainable, dtype=dtype, **kwargs)
        # The first layer transforms the complex points to the bihomogeneous form.
        # The number of the outputs is d^2, where d is the number of coordinates.
        self.bihomogeneous = bnn.Bihomogeneous(d=len(Z))
        self.layer1 = bnn.SquareDense(len(Z)**2, 70, activation=tf.square)
        self.layer2 = bnn.SquareDense(70, 100, activation=tf.square)
        self.layer3 = bnn.SquareDense(100, 1)

    def call(self, inputs):
        x = self.bihomogeneous(inputs)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = tf.math.log(x)
        return x

In [ ]:
model = Kahler_potential()

Define the Kähler metric $g_{i \bar j} = \partial_i\bar\partial_{\bar j} K$ and the volume form $d\mu_g = \det g_{i \bar j}$:

In [ ]:
@tf.function
def volume_form(points, Omega_Omegabar, mass, restriction):

    kahler_metric = mlg.complex_math.complex_hessian(tf.math.real(model(points)), points)
    kahler_metric = tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True))
    det_g = tf.math.real(tf.linalg.det(kahler_metric))

    # Calculate the normalization constant to make the overall integration as 1
    # It is a batchwise calculation but we expect it to converge to a constant eventually
    # Consequently, if one computes the average of volume_form / Omega_Omegabar,
    # they will get strictly 1. (Actually the result would be Vol_Omega, but we set
    # it to be 1 here implicitly.)
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * det_g / Omega_Omegabar)
    volume_form = det_g / factor

    return volume_form

### Train the model with Adam and L-BFGS
#### Adam
Setup the keras optmizer as `Adam` and the loss function as one of weighted loss in the `mlg.loss` module. Some available functions are `weighted_MAPE`, `weighted_MSE`, `max_error` and `MAPE_plus_max_error`. They are weighted with the mass formula since the points on the hypersurface are distributed according to the Fubini-Study measure while the measure used in the integration is determined by the volume form $\small \Omega \wedge \bar\Omega$.

In [ ]:
optimizer = keras.optimizers.Adam()
loss_func = mlg.loss.weighted_MAPE

Loop over the batches and train the network:

In [ ]:
max_epochs = 50
epoch = 0
while epoch < max_epochs:
    epoch = epoch + 1
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(train_set):
        with tf.GradientTape() as tape:
            det_omega = volume_form(points, Omega_Omegabar, mass, restriction)
            loss = loss_func(Omega_Omegabar, det_omega, mass)
            grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    if epoch % 10 == 0:
        print("epoch %d: loss = %.5f" % (epoch, loss))
        #print("%.5f" % (loss))


epoch 10: loss = 0.00413
epoch 20: loss = 0.00361
epoch 30: loss = 0.00258
epoch 40: loss = 0.00293
epoch 50: loss = 0.00274


Let's check the loss of the test dataset. First define a function to calculate the total loss over the whole dataset:

In [ ]:
def cal_total_loss(dataset, loss_function):
    total_loss = tf.constant(0, dtype=tf.float32)
    total_mass = tf.constant(0, dtype=tf.float32)

    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(dataset):
        det_omega = volume_form(points, Omega_Omegabar, mass, restriction)
        mass_sum = tf.reduce_sum(mass)
        total_loss += loss_function(Omega_Omegabar, det_omega, mass) * mass_sum
        total_mass += mass_sum
    total_loss = total_loss / total_mass

    return total_loss.numpy()

Check the results of MAPE and MSE:

In [ ]:
sigma_test = cal_total_loss(test_set, mlg.loss.weighted_MAPE)
E_test = cal_total_loss(test_set, mlg.loss.weighted_MSE)
print("sigma_test = %.5f" % sigma_test)
print("E_test = %.5f" % E_test)

sigma_test = 0.00320
E_test = 0.00002


You can also check the error of the Monte Carlo integration, estimated by:

$$\delta \sigma = \frac{1}{\sqrt{N_p}} {\left( \int_X (|\eta - 1_X| - \sigma)^2 d\mu_{\Omega}\right)}^{1/2},$$

where $N_p$ is the number of points on the hypersurface and $\sigma$ is the `weighted_MAPE` loss, and

$$\eta = \frac{\det \omega}{\small \Omega \wedge \bar \Omega}$$

In [ ]:
def delta_sigma_square_test(y_true, y_pred, mass):
    weights = mass / tf.reduce_sum(mass)
    return tf.reduce_sum((tf.abs(y_true - y_pred) / y_true - sigma_test)**2 * weights)

delta_sigma = cal_total_loss(test_set, delta_sigma_square_test)
print("delta_simga = %.5f" % delta_sigma)

delta_simga = 0.00001


In [ ]:
@tf.function
def volume_form_with_model(points, Omega_Omegabar, mass, restriction, this_model):

    kahler_metric = mlg.complex_math.complex_hessian(tf.math.real(this_model(points)), points)
    kahler_metric = tf.matmul(restriction, tf.matmul(kahler_metric, restriction, adjoint_b=True))
    volume_form = tf.math.real(tf.linalg.det(kahler_metric))

    # Calculate the normalization constant to make the overall integration as 1
    # It is a batchwise calculation but we expect it to converge to a constant eventually
    weights = mass / tf.reduce_sum(mass)
    factor = tf.reduce_sum(weights * volume_form / Omega_Omegabar)

    # Frank's modification
    #print(tf.executing_eagerly())

    """
    N   = tf.shape(kahler_metric)[0]
    idx = tf.random.shuffle(tf.range(N))[:100]
    M   = tf.gather(kahler_metric, idx)

    for i in tf.range(tf.shape(M)[0]):
        tf.print("matrix", i, idx[i], "\n real:\n", tf.math.real(M[i]),
                 "\n imag:\n", tf.math.imag(M[i]), summarize=-1)
    """

    file_name = 'metric_ellptic_xt.csv'

    with open(file_name,'w',newline='') as f:
        writer = csv.writer(f)
        #writer.writerow(['Point Index','Points','Kahler Metric (Real)','Kahler Metric (Imag)'])
        for i in range(120):
            #tf.print("matrix", i, pts[i], tf.math.real(kahler_metric[i]), tf.math.imag(kahler_metric[i]), summarize=-1)
            #tf.print(i, "points=", pts[i], "Kahler_metric_real=", tf.math.real(kahler_metric[i]), "Kahler_metric_imag=", tf.math.imag(kahler_metric[i]), summarize=-1,output_stream="file://"+file_name)
            tf.print([i, ";", pts[i], ";", tf.math.real(kahler_metric[i]), ";", tf.math.imag(kahler_metric[i])], summarize=-1, output_stream="file://"+file_name)
    print(f"Done! Results in {file_name}")

    """
    for i in range(120):
        tf.print("matrix", i, pts[i], tf.math.real(kahler_metric[i]), tf.math.imag(kahler_metric[i]), summarize=-1)



    # Convert 2x2 complex to 4x4 real Hermitian?
    # See "Riemannian metric.ipynb"

    #eigvals = tf.linalg.eigh(M)[0]
    #eigvals_np = eigvals.numpy()
    """
    return volume_form / factor

In [ ]:
class Fubini_potential(tf.keras.Model):
    def __init__(self, trainable=True, dtype='float32', **kwargs):
        super(Fubini_potential, self).__init__(trainable=trainable, dtype=dtype, **kwargs)
        self.abs_layer = tf.keras.layers.Lambda(lambda x: tf.abs(x))
        self.layer1 = bnn.SquareDense(len(Z), len(Z), activation=tf.square)
        self.layer2 = bnn.SquareDense(len(Z), 1)

    def call(self, inputs):
        x = self.abs_layer(inputs)  # Convert complex64 to float32 by taking the absolute value
        x = self.layer1(x)
        x = self.layer2(x)
        x = tf.math.log(x)
        return x

fubi = Fubini_potential()
fubi.layer1.w.assign(tf.constant([[1,0,0],[0,1,0],[0,0,1]], dtype=tf.float32))
fubi.layer2.w.assign(tf.constant([[1],[1],[1]], dtype=tf.float32))
"""
fubi.layer1.w.assign(tf.constant([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], dtype=tf.float32))
fubi.layer2.w.assign(tf.constant([[1],[1],[1],[1]], dtype=tf.float32))
"""

'\nfubi.layer1.w.assign(tf.constant([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]], dtype=tf.float32))\nfubi.layer2.w.assign(tf.constant([[1],[1],[1],[1]], dtype=tf.float32))\n'

In [ ]:
def compute_quotient(y_numerator, y_denominator, mass):  # mass-weighted
    weights = mass / tf.reduce_sum(mass)
    return tf.reduce_sum(tf.abs(y_numerator / y_denominator) * weights)

def calculate_volume(dataset):
    total_holo_volume = tf.constant(0, dtype=tf.float32)
    total_kaehler_volume = tf.constant(0, dtype=tf.float32)
    total_fubi_volume = tf.constant(0, dtype=tf.float32)
    total_mass = tf.constant(0, dtype=tf.float32)
    num_points = 0
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(dataset):
        det_fubi = volume_form_with_model(points, Omega_Omegabar, mass, restriction, fubi)
        det_omega = volume_form_with_model(points, Omega_Omegabar, mass, restriction, model)
        mass_sum = tf.reduce_sum(mass)
        total_holo_volume += compute_quotient(Omega_Omegabar, det_fubi, mass) * mass_sum
        total_kaehler_volume += compute_quotient(det_omega, det_fubi, mass) * mass_sum
        total_fubi_volume += compute_quotient(det_fubi, det_fubi, mass)
        total_mass += mass_sum
        num_points += len(points)
    total_holo_volume = total_holo_volume / total_mass
    total_kaehler_volume = total_kaehler_volume / total_mass

    return num_points, total_mass.numpy(), total_holo_volume.numpy(), total_kaehler_volume.numpy(), total_fubi_volume.numpy()

num_points, total_mass, holo_vol, kaehler_vol, fubi_volume = calculate_volume(test_set)
print(f'Num points: {num_points}; Total mass: {total_mass}; Holomorphic volume: {holo_vol}; Kahler volume: {kaehler_vol}; Fubi volume: {fubi_volume}')

Done! Results in metric_ellptic_xt.csv
Done! Results in metric_ellptic_xt.csv
Num points: 3840; Total mass: 17358.966796875; Holomorphic volume: 1.0306395292282104; Kahler volume: 1.0304555892944336; Fubi volume: 30.0


In [ ]:
def compute_quotient(y_numerator, y_denominator, mass): # batch-average
    weights = mass / tf.reduce_sum(mass)
    return tf.reduce_sum(tf.abs(y_numerator / y_denominator) * weights)

def calculate_volume(dataset):
    total_holo_volume = tf.constant(0, dtype=tf.float32)
    total_kaehler_volume = tf.constant(0, dtype=tf.float32)
    total_fubi_volume = tf.constant(0, dtype=tf.float32)
    total_mass = tf.constant(0, dtype=tf.float32)
    num_points = 0
    for step, (points, Omega_Omegabar, mass, restriction) in enumerate(dataset):
        det_fubi = volume_form_with_model(points, Omega_Omegabar, mass, restriction, fubi)
        det_omega = volume_form_with_model(points, Omega_Omegabar, mass, restriction, model)
        mass_sum = tf.reduce_sum(mass)
        total_holo_volume += compute_quotient(Omega_Omegabar, Omega_Omegabar, mass)
        total_kaehler_volume += compute_quotient(det_omega, Omega_Omegabar, mass)
        total_fubi_volume += compute_quotient(det_fubi, Omega_Omegabar, mass)
        total_mass += mass_sum
        num_points += len(points)
    total_holo_volume = total_holo_volume / total_mass
    total_kaehler_volume = total_kaehler_volume / total_mass

    return num_points, total_mass.numpy(), total_holo_volume.numpy(), total_kaehler_volume.numpy(), total_fubi_volume.numpy()

num_points, total_mass, holo_vol, kaehler_vol, fubi_volume = calculate_volume(test_set)
print(f'Num points: {num_points}; Total mass: {total_mass}; Holomorphic volume: {holo_vol}; Kahler volume: {kaehler_vol}; Fubi volume: {fubi_volume}')

Num points: 3840; Total mass: 17358.96875; Holomorphic volume: 0.001728213275782764; Kahler volume: 0.001728213275782764; Fubi volume: 30.0


In [ ]:
files.download('metric_ellptic_xt.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# (IRRELEVANT)
Let's see if the computation matches our manual computation of the ratio, at point q = (1, exp(i pi/4), 0, 0),

The manually calculated ratio is 1/16 (or i/16 up to a factor of i/2pi)

In [ ]:
@tf.function
def ratio_in_batch(points, Omega_Omegabar, mass, restriction):
    """
    Returns a Tensor of shape (batch_size,) with
    ratio[i] = Omega_Omegabar[i] / det(model's metric)[i].
    This uses the raw determinant before any normalization is applied.
    """
    kahler_metric = mlg.complex_math.complex_hessian(tf.math.real(model(points)), points)
    kahler_metric = tf.matmul(
        restriction,
        tf.matmul(kahler_metric, restriction, adjoint_b=True)
    )

    det_kahler = tf.math.real(tf.linalg.det(kahler_metric))

    ratio = Omega_Omegabar / det_kahler

    return ratio

In [ ]:
all_ratios = []
for step, (points, Omega_Omegabar, mass, restriction) in enumerate(test_set):
    batch_ratio = ratio_in_batch(points, Omega_Omegabar, mass, restriction)
    all_ratios.append(batch_ratio)

all_ratios = tf.concat(all_ratios, axis=0)

all_ratios_np = all_ratios.numpy()

print("Ratio array shape:", all_ratios_np.shape)

Ratio array shape: (3840,)


In [ ]:
q_np = np.array([1.0, 0.0, 0.0], dtype=np.complex64)

THRESHOLD = 0.45
all_close_ratios = []

for step, (points, Omega_Omegabar, mass, restriction) in enumerate(test_set):
    points_np = points.numpy()  # shape (batch_size, 4) if CP^3

    dist = np.linalg.norm(points_np - q_np, axis=1)

    mask = dist < THRESHOLD

    if not np.any(mask):
        continue

    batch_ratios = ratio_in_batch(points, Omega_Omegabar, mass, restriction)
    batch_ratios_np = batch_ratios.numpy()

    close_ratios = batch_ratios_np[mask]
    all_close_ratios.append(close_ratios)

if len(all_close_ratios) == 0:
    print("No points found within distance", THRESHOLD)
else:
    all_close_ratios = np.concatenate(all_close_ratios, axis=0)
    print(f"Found {len(all_close_ratios)} points within distance {THRESHOLD}.")
    print("Mean ratio near q:", all_close_ratios.mean())
    print("Std dev ratio near q:", all_close_ratios.std())

Found 86 points within distance 0.45.
Mean ratio near q: 0.56973535
Std dev ratio near q: 0.0015814168
